<a href="https://colab.research.google.com/github/chaitri19/Advance-Cricket-Simulation/blob/main/AdvancedCricketSimulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import random

In [12]:
prob_noball=0.1
prob_wideball=0.05
prob_catch=0.2
prob_lbw=0.15

In [13]:
class Player:
  def __init__(self,name,bowling,batting,fielding,running,experience):
    self.name=name
    self.bowling=bowling
    self.batting=batting
    self.fielding=fielding
    self.running=running
    self.experience=experience

In [14]:
class Team:
  def __init__(self,name,players):
    self.name=name
    self.players=players
    self.caption=None
    self.batting_order=players.copy()
    self.bowlers=[]

  def caption_selection(self,caption):
    self.caption=caption

  def send_next_player(self):
    if(len(self.batting_order)>0):
      return self.batting_order.pop(0)
    else:
      return None

  def choose_bowler(self):
    return random.choice(self.bowlers)

In [15]:
class Field:
  def __init__(self,size,fan_ratio,pitch_condition,home_advantage):
    self.size=size
    self.fan_ratio=fan_ratio
    self.pitch_condition=pitch_condition
    self.home_advantage=home_advantage

In [16]:
class Umpire:
  def __init__(self,field):
    self.field=field
    self.score=0
    self.wicket=0
    self.over=0

  def update_score(self,runs):
    self.score=self.score+runs

  def update_wicket(self):
    self.wicket=self.wicket+1

  def update_over(self):
    self.over=self.over+1

  def predict_result(self,batsman,bowler):
    prob_batting=batsman.batting * self.field.pitch_condition * random.random()
    prob_bowling=bowler.bowling * self.field.pitch_condition * random.random()
    if prob_batting>prob_bowling:
      if(random.random()<prob_catch):
        print("Catch")
        self.update_wicket()
      elif(random.random()<prob_lbw):
        print("LBW!")
        self.update_wicket()
      return "OUT"
    else:
      if(random.random()<prob_noball):
        print("No Ball")
        self.update_score(1)
      elif(random.random()<prob_wideball):
        print("Wide Ball")
        self.update_score(1)
      return "NOT OUT"

In [17]:
class Commentator:
  def __init__(self,umpire):
    self.umpire=umpire

  def describe_ball(self,batsman,bowler):
    res=self.umpire.predict_result(batsman,bowler)
    print("Result: ",res)
    if res=="OUT":
      say= batsman.name + " is OUT!"
    else:
      say= batsman.name + " plays the shot."
    return say

  def describe_game(self,caption1,caption2,country1,country2,over):
    print("\nLet's start the game. \n")
    print("Today we have "+country1+" Vs "+country2)
    print("So our first caption is "+caption1+" and our second caption is "+caption2)
    print("Number of overs is: "+ str(over))

  def describe_start(self,team):
      print("\nGame Started \n")
      print("Currently team "+team+" is playing:")

  def describe_end(self):
      print("\n So the final score is "+str(self.umpire.score)+" with "+str(self.umpire.wicket)+" wickets in "+str(self.umpire.over)+" overs")

  def current_info(self,ball_count):
      print("Currently "+str(ball_count)+" balls are used with "+str(self.umpire.over)+" overs securing "+str(self.umpire.score)+" runs in "+str(self.umpire.wicket)+" wickets")

  def describe_final_result(self,name,score):
      print("\nAnd the winner of the match is:")
      print("Team:"+name+" with score:"+str(score))


In [18]:
class Match:
  def __init__(self,team1,team2,field,total_overs):
    self.team1=team1
    self.team2=team2
    self.field=field
    self.umpire=Umpire(field)
    self.commentator=Commentator(self.umpire)
    self.total_overs=total_overs

  def start_match(self):
    self.team1.caption_selection(random.choice(self.team1.players))
    self.team2.caption_selection(random.choice(self.team2.players))
    self.team1.batting_order=self.team1.players.copy()
    self.team2.batting_order=self.team2.players.copy()
    self.team1.bowlers=self.team1.players.copy()
    self.team2.bowlers=self.team2.players.copy()
    self.commentator.describe_game(self.team1.caption.name, self.team2.caption.name,self.team1.name,self.team2.name,self.total_overs)

    #When team 1 is playing
    self.commentator.describe_start(self.team1.name)
    self.play_innings(self.team1, self.team2)
    self.commentator.describe_end()
    lastscore=self.commentator.umpire.score

    #When team 2 is playing
    self.commentator.umpire.score=0
    self.commentator.umpire.wicket=0
    self.commentator.umpire.over=0
    self.commentator.describe_start(self.team2.name)
    self.play_innings(self.team2,self.team1)
    self.commentator.describe_end()
    newscore=self.commentator.umpire.score

    if lastscore > newscore:
      self.commentator.describe_final_result(team1.name, lastscore)
    else:
      self.commentator.describe_final_result(team2.name, newscore)

  def play_innings(self,batting_team,bowling_team):
      ball_count=1
      over=0
      bowler=bowling_team.choose_bowler()
      batsman=batting_team.send_next_player()

      while over < self.total_overs:
        self.commentator.current_info(ball_count)
        ball_description=self.commentator.describe_ball(batsman,bowler)
        print(ball_description)
        if ball_description.endswith("OUT!"):
          batsman=batting_team.send_next_player()
          if batsman is None:
            break
          self.umpire.update_wicket()
          print("Wickets:"+str(self.umpire.wicket)+" , Overs:"+str(self.umpire.over))
          print("New player "+batsman.name+" is playing...")
        else:
          runs=random.randint(0,6)
          self.umpire.update_score(runs)

        if ball_count > 5:
          over += 1
          print("Over "+str(over)+" is starting now!")
          self.umpire.update_over()
          bowler=bowling_team.choose_bowler()
          ball_count=0

        self.commentator.current_info(ball_count)
        ball_count=ball_count+1


In [19]:
player1=[]
for i in range(10):
  player1.append(Player("Player1_"+str(i+1),round(random.random(),1),round(random.random(),1),round(random.random(),1),round(random.random(),1),round(random.random(),1)))

player2=[]
for i in range(10):
  player2.append(Player("Player2_"+str(i+1),round(random.random(),1),round(random.random(),1),round(random.random(),1),round(random.random(),1),round(random.random(),1)))


team1=Team("Country1",player1)
team2=Team("Country2",player2)

field=Field("Large",0.7,0.8,0.9)

total_overs=2
match=Match(team1,team2,field,total_overs)
match.start_match()



Let's start the game. 

Today we have Country1 Vs Country2
So our first caption is Player1_7 and our second caption is Player2_2
Number of overs is: 2

Game Started 

Currently team Country1 is playing:
Currently 1 balls are used with 0 overs securing 0 runs in 0 wickets
Result:  OUT
Player1_1 is OUT!
Wickets:1 , Overs:0
New player Player1_2 is playing...
Currently 1 balls are used with 0 overs securing 0 runs in 1 wickets
Currently 2 balls are used with 0 overs securing 0 runs in 1 wickets
Result:  OUT
Player1_2 is OUT!
Wickets:2 , Overs:0
New player Player1_3 is playing...
Currently 2 balls are used with 0 overs securing 0 runs in 2 wickets
Currently 3 balls are used with 0 overs securing 0 runs in 2 wickets
Result:  OUT
Player1_3 is OUT!
Wickets:3 , Overs:0
New player Player1_4 is playing...
Currently 3 balls are used with 0 overs securing 0 runs in 3 wickets
Currently 4 balls are used with 0 overs securing 0 runs in 3 wickets
Result:  OUT
Player1_4 is OUT!
Wickets:4 , Overs:0
New 